In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import glob

In [2]:
def avantes_aggregator(filepath,fileout):
    """ Aggregates Avantes ASCII 2D spectra takes the log folder (avantes generated) and the name
    (with path) of the output npy """
    files = glob.glob(filepath+'*.TXT')
    #print(files)
    for ii,filename in enumerate(files):
        counts = []
        if ii==0:
            wavelength = []
            with open(filename) as file:
                lineas=file.readlines()
                lineas=lineas[8:-2]
            for line in lineas:
                lines=line.split(';')
                wavelength.append(float(lines[0]))
                counts.append(float(lines[1]))
            spectra=np.concatenate((np.array(wavelength,ndmin=2).T,np.array(counts,ndmin=2).T),axis=1)
        else:
            with open(filename) as file:
                lineas=file.readlines()
                lineas=lineas[8:-2]
            for line in lineas:
                lines=line.split(';')
                counts.append(float(lines[1]))
            spectra=np.concatenate((spectra,np.array(counts,ndmin=2).T),axis=1)
    np.save(fileout,spectra)

def spectra_accumulator(npyin,samples,verbose=0):
    """ Accumulates a number of samples from the single spectrum npy matrix """
    ceaspec = np.load(npyin)
    cols=len(ceaspec[0,:])

    fromcol = 1
    tocol = fromcol + samples
    accum=np.copy(ceaspec[:,0]).reshape(len(ceaspec[:,0]),1)

    while tocol<=cols:
        #print(fromcol,tocol)
        suma=np.sum(ceaspec[:,fromcol:tocol],axis=1,keepdims=1)
        accum=np.concatenate((accum,suma),axis=1)
        tocol=tocol+samples
        fromcol=fromcol+samples
    if verbose==1:
        print(accum.shape)
    return accum

def extinction(I_sample, I_0, Reff, distance):
    "Calculates extinction spectrum"
    I_ratio=(I_0/I_sample)
    return (1/distance)*(I_ratio-1)*(1-Reff)

In [24]:
##### Spectra handling

In [4]:
## Load files into matrices
logfolder1 = 'D:\\CCA\\Avantes\\logs29nov\\'
#logfolder1 = '/home/mixtli/Avantes/logs30nov/'
#DCbg1=np.load(logfolder1 + 'CEASbckg1DC.npy')
#DCbg2=np.load(logfolder1 + 'CEASbckg2DC.npy')
#DCln1=np.load(logfolder1 + 'CEASline1DC.npy')

SCbg1 = spectra_accumulator(logfolder1 + 'CEASbckg1SC1.npy',20)
SCbg2 = spectra_accumulator(logfolder1 + 'CEASbckg2SC1.npy',20)
#SCln1 = spectra_accumulator(logfolder1 + 'CEASline1SC1.npy',20)

In [ ]:
## Extinction
line=SCln1
bckg=SCbg2
I_sample=np.average(line[:,1:],axis=1)
I_0=np.average(bckg[:,1:],axis=1)
alpha1 = extinction(I_sample,I_0,0.99985,70)

line=SCln1
bckg=SCbg1
I_sample=np.average(line[:,1:],axis=1)
I_0=np.average(bckg[:,1:],axis=1)
alpha2 = extinction(I_sample,I_0,0.99985,70)